In [6]:
import os
import re
import gzip
import json
import matplotlib.pyplot as plt
# name of channel to be scanned
channel_prefix = "https://www.youtube.com/"
#channel = "@StronnyCuttles"
#channel = "@ShibiCottonbum"
channel = "@ImmyBisou"
#channel = "@IceySnowpaws"
#channel = "@MercyModiste"
#channel = "@AzuraDulait"

In [7]:
try:   os.mkdir(channel)
except:   pass
try:   os.mkdir(f"{channel}/downloaded")
except:   pass
try:   os.mkdir(f"{channel}/processed")
except:   pass
try:   os.mkdir(f"{channel}/transcripts")
except:   pass
try:  os.mkdir(f"{channel}/chats")
except:  pass

# file containing keywords
keywords_file = "keywords.txt"

# file containing thematic searches
thematic_file = "thematic.txt"

# scan channel for videos
# live videos
os.system(f"yt-dlp --get-id --skip-download {channel_prefix+channel} > {channel}/videos.txt")
videos = []
with open(f"{channel}/videos.txt", "r") as f:
    for line in f:
        if line.startswith("Error"):
            continue
        videos.append(line.strip())

# look for filenames in {channel}/downloaded for already downloaded videos
# the file name contains the video id in the following format:
# 【Supermarket Simulator】Definitely not drunk on the job 🐐🍼【VAllure】 [i3rrDSjokF4].webm
# where [i3rrDSjokF4] is the video id
downloaded = []
for file in os.listdir(f"{channel}/downloaded"):
    if file.endswith(".webm"):
        m = re.search(r"\[(.*?)\]", file)
        if m:
            downloaded.append(m.group(1))
            
# do the same for transcripts
transcripts = []
for file in os.listdir(f"{channel}/transcripts"):
    if file.endswith(".srt"):
        m = re.search(r"\[([^\[\]]+)\](?!.*\[)", file)
        if m:
            transcripts.append(m.group(1))


# compare list of videos to videos already downloaded
to_download = list(set(videos) - set(downloaded) - set(transcripts))
for download in to_download:
    print(f"{download}")

xpXHpulP3t8
afEcoyCO0ic
qc5NXD5C468
tdPbjUgTNUY
n9pVE623KDc
dlVp_rafsPQ
HIqizrP-Bao
vEO60rvWG-I
Agh-eyrFSvw
xcFd6vK6NP0
Ixqs3zZ6wZQ
qXmPafBM59c
iGCSAKseNBQ
gif6RfYixSs
NM8iMXxKErI
tYTIkFPtS0k
kz5YO52NshU
OYk68iP-3U8
37KDHSUIeVA
n57EYjU9NCE
GkwWfegi8ok
9KjTu3tkcVE
JcZy5ChDIDQ
rPpvWB8ayyw
YwIwcrciVDs
9crWJZU_rb0
yOmox5i6u7w
eRwNZ5iQ0uw


In [8]:
newly_downloaded = []
for video in to_download:
    print(f"Downloading {video}")
    #output = os.system(f"yt-dlp -f bestaudio -P {channel}/downloaded {channel_prefix}watch?v={video}")
    output = os.system(f"yt-dlp --write-subs --sub-lang live_chat -f bestaudio -P {channel}/downloaded {channel_prefix}watch?v={video}")
    if output!=0:
        print(f"Error downloading {video}")

In [9]:
#collect file names in {channel}/downloaded
downloaded_files = os.listdir(f"{channel}/downloaded")
for file in downloaded_files:
    if file.endswith(".webm") or file.endswith(".m4a"):
        #result = os.system(f"whisper \"{channel}/downloaded/{file}\" --device cuda --output_dir --language English --model large-v3 \"{channel}/transcripts/\"")
        result = os.system(f"whisper \"{channel}/downloaded/{file}\" --device cuda --language English --output_dir \"{channel}/transcripts/\"")
        if result==0:
            #move file to {channel}/processed
            try:
                os.rename(f"{channel}/downloaded/{file}", f"{channel}/processed/{file}")
            except:
                print(f"Error moving {file} to {channel}/processed")

In [10]:
# move and transform chat scripts. we want to extract user, timestamp and message
# chat scripts are in the format:
#{"clickTrackingParams": "CAEQl98BIhMI0Mqgo6G_iAMV-JzCAR2cWziB", "replayChatItemAction": {"actions": [{"clickTrackingParams": "CAEQl98BIhMI0Mqgo6G_iAMV-JzCAR2cWziB", "addChatItemAction": {"item": {"liveChatTextMessageRenderer": {"message": {"runs": [{"emoji": {"emojiId": "🩷", "shortcuts": [":pink_heart:"], "searchTerms": ["pink", "heart"], "image": {"thumbnails": [{"url": "https://fonts.gstatic.com/s/e/notoemoji/15.1/1fa77/72.png"}], "accessibility": {"accessibilityData": {"label": "🩷"}}}}}, {"emoji": {"emojiId": "🤍", "shortcuts": [":white_heart:"], "searchTerms": ["white", "heart"], "image": {"thumbnails": [{"url": "https://fonts.gstatic.com/s/e/notoemoji/15.1/1f90d/72.png"}], "accessibility": {"accessibilityData": {"label": "🤍"}}}}}, {"emoji": {"emojiId": "🤎", "shortcuts": [":brown_heart:"], "searchTerms": ["brown", "heart"], "image": {"thumbnails": [{"url": "https://fonts.gstatic.com/s/e/notoemoji/15.1/1f90e/72.png"}], "accessibility": {"accessibilityData": {"label": "🤎"}}}}}, {"emoji": {"emojiId": "❤", "shortcuts": [":red_heart:", ":heart:", "<3"], "searchTerms": ["red", "heart"], "image": {"thumbnails": [{"url": "https://fonts.gstatic.com/s/e/notoemoji/15.1/2764/72.png"}], "accessibility": {"accessibilityData": {"label": "❤"}}}}}]}, "authorName": {"simpleText": "TappedPotential🐰👠"}, "authorPhoto": {"thumbnails": [{"url": "https://yt4.ggpht.com/mAj0LNN_j5Bv3kr0iTP6bNsE-BIwkoUXCqIOR1zahwcF7Q46xBzVp3JrCuBkwGhRRmO8_0YOlA=s32-c-k-c0x00ffffff-no-rj", "width": 32, "height": 32}, {"url": "https://yt4.ggpht.com/mAj0LNN_j5Bv3kr0iTP6bNsE-BIwkoUXCqIOR1zahwcF7Q46xBzVp3JrCuBkwGhRRmO8_0YOlA=s64-c-k-c0x00ffffff-no-rj", "width": 64, "height": 64}]}, "contextMenuEndpoint": {"clickTrackingParams": "CAEQl98BIhMI0Mqgo6G_iAMV-JzCAR2cWziB", "commandMetadata": {"webCommandMetadata": {"ignoreNavigation": true}}, "liveChatItemContextMenuEndpoint": {"params": "Q2g0S0hBb2FRMHRwV1RCMlRHWnlOR2RFUmxSclN6Rm5RV1JJVW1OSlFWRWFLU29uQ2hoVlEzWlplbmRGYTBOUk1qZG5VR0ZxTlRSM1FuTldUMmNTQzNSek5qaDFRelF0WTBrMElBRW9BVElhQ2hoVlF6bDBjVjlpYlVSbU9YYzVNblpFYm1ReWIwdzJNbWM0QWtnQlVBRSUzRA=="}}, "id": "ChwKGkNLaVkwdkxmcjRnREZUa0sxZ0FkSFJjSUFR", "timestampUsec": "1725674881850547", "authorBadges": [{"liveChatAuthorBadgeRenderer": {"customThumbnail": {"thumbnails": [{"url": "https://yt3.ggpht.com/LROENIDC2LQf2oAC1WcICEsPbihMHhp5APeKA9jks871KM4K8lrSQN_uoK85gmE3c8qmjFc96Q=s16-c-k", "width": 16, "height": 16}, {"url": "https://yt3.ggpht.com/LROENIDC2LQf2oAC1WcICEsPbihMHhp5APeKA9jks871KM4K8lrSQN_uoK85gmE3c8qmjFc96Q=s32-c-k", "width": 32, "height": 32}]}, "tooltip": "Member (2 months)", "accessibility": {"accessibilityData": {"label": "Member (2 months)"}}}}], "authorExternalChannelId": "UC9tq_bmDf9w92vDnd2oL62g", "contextMenuAccessibility": {"accessibilityData": {"label": "Chat actions"}}, "timestampText": {"simpleText": "-8:48"}, "trackingParams": "CAEQl98BIhMI0Mqgo6G_iAMV-JzCAR2cWziB"}}, "clientId": "CKiY0vLfr4gDFTkK1gAdHRcIAQ"}}], "videoOffsetTimeMsec": "0"}}
#show debug if:
# has more than 1 action
# has more than 1 run
# doesn't have user
# videoOffsetTimeMsec is lower than 0 or bigger than 7200000 (max 12 hours)
# doesn't have message
files = os.listdir(f"{channel}/downloaded")
chat_files = [i for i in files if i.endswith("live_chat.json")]
#print(chat_files)
for chat_file in chat_files:
    all_timestamps = []
    window_size_seconds = 60
    
    with open(os.path.join(channel, "downloaded", chat_file), 'r', encoding='utf-8') as file, open(os.path.join(channel, "chats", f"{chat_file[:-15]}.txt"), 'w', encoding='utf-8') as output_file:
        #try:
            for line in file:
                line = line.strip()
                try:
                    json_data = json.loads(line)
                except json.JSONDecodeError:
                    print(f"Invalid JSON: {line}")
                    continue

                # Extract timestamp
                replay_action = json_data.get('replayChatItemAction')
                if not replay_action:
                    print(f"No replayChatItemAction: {line}")
                    continue

                video_offset_time_msec = replay_action.get('videoOffsetTimeMsec')
                if not video_offset_time_msec:
                    print(f"No videoOffsetTimeMsec: {line}")
                    continue

                try:
                    timestamp_ms = int(video_offset_time_msec)
                except ValueError:
                    print(f"Invalid videoOffsetTimeMsec value: {line}")
                    continue

                # Validate timestamp
                if timestamp_ms < 0 or timestamp_ms > 43200000:
                    print(f"Invalid timestamp: {line}")
                    continue

                # Process each action
                actions = replay_action.get('actions', [])
                for action in actions:
                    add_chat_item_action = action.get('addChatItemAction')
                    if not add_chat_item_action:
                        continue

                    item = add_chat_item_action.get('item', {})
                    live_chat_renderer = item.get('liveChatTextMessageRenderer')
                    if not live_chat_renderer:
                        continue

                    # Extract user
                    author_name = live_chat_renderer.get('authorName', {})
                    user = author_name.get('simpleText', '')
                    if not user:
                        user = live_chat_renderer['authorExternalChannelId']
                    # if not user:
                    #     print(f"Invalid user: {line}")
                    #     continue

                    # Extract message
                    message = ''
                    message_runs = live_chat_renderer.get('message', {}).get('runs', [])
                    for run in message_runs:
                        text = run.get('text')
                        if text:
                            message += text
                        else:
                            emoji = run.get('emoji', {})
                            shortcuts = emoji.get('shortcuts', [])
                            if len(shortcuts) > 0:
                                emoji_id = shortcuts[0]
                            else:
                                emoji_id = emoji.get('emojiId', '') # This is the actual emoji. sometimes with claps, it doesn't have shortcuts
                                if not emoji_id:
                                    print(f"whack issue with emoji for line: {line}")
                            
                            message += emoji_id

                    if not message:
                        print(f"Invalid message: {line}")
                        continue
                    
                    message = message.replace("\n", " ").replace("\t", " ")
                    # Process the extracted data (e.g., print or write to a file)
                    output_file.write(f"{user}\t{timestamp_ms}\t{message}\n")

        # except Exception as e:
        #     print(f"Unknown error: {e}")
            
    # remove chat file
    os.remove(os.path.join(channel, "downloaded", chat_file))

        

    

In [11]:
#remove .tsv, .txt, .vtt files, (keeping .srt and .json), and compress json files to .json.gz in {channel}/transcripts
transcript_files = os.listdir(f"{channel}/transcripts")
for file in transcript_files:
    if file.endswith(".tsv") or file.endswith(".txt") or file.endswith(".vtt"):
        try:
            os.remove(f"{channel}/transcripts/{file}")
        except:
            print(f"Error removing {file}")
    if file.endswith(".json"):
        with open(f"{channel}/transcripts/{file}", "rb") as f:
            with gzip.open(f"{channel}/transcripts/{file}.gz", "wb") as f_out:
                f_out.writelines(f)
        #remove original json file
        try:
            os.remove(f"{channel}/transcripts/{file}")
        except:
            print(f"Error removing {file}")
        